# LRU算法

## 使用 OrderedDict 实现

In [1]:
from collections import OrderedDict, abc

In [2]:
class LRUCacheV1(abc.MutableMapping):
    """使用OrderedDict实现的 LRU cache"""
    
    def __init__(self, size, *args, **kwargs):
        self.size = size
        # 用 OrderedDict 保存，最近访问过的放在尾部
        self._map = OrderedDict(*args, **kwargs)

    def __setitem__(self, key, value):
        assert value is not None, "value cannot be None"

        self._map[key] = value
        self._map.move_to_end(key)  # 将最新的放在尾部
        if len(self._map) > self.size:
            self._map.popitem(last=False)  # pop first item

    def __getitem__(self, key):
        value = self._map.get(key)
        if value is None:  # key not exist
            return value

        self._map.move_to_end(key)
        return value

    def __delitem__(self, key):
        self._map.pop(key)

    def __len__(self):
        return len(self._map)

    def __iter__(self):
        for k, v in reversed(self._map.items()):
            yield '<%s=%s>' % (k, v)

    def __contains__(self, key):
        return key in self._map

    def __str__(self):
        return '<LRUCache1 len=%s %s>' % (len(self._map), list(self))

In [3]:
lru_cache1 = LRUCacheV1(5)
lru_cache1['a'] = 1
lru_cache1['b'] = 2
lru_cache1['c'] = 3
lru_cache1['d'] = 4

print('len=%s' % len(lru_cache1))
print(lru_cache1)

len=4
<LRUCache1 len=4 ['<d=4>', '<c=3>', '<b=2>', '<a=1>']>


In [4]:
print('a=%s' % lru_cache1.get('a'))
print(lru_cache1)  # 'a' moved to the first

a=1
<LRUCache1 len=4 ['<a=1>', '<d=4>', '<c=3>', '<b=2>']>


In [5]:
lru_cache1['e'] = 5
print(lru_cache1)  # now meet the max size=5

<LRUCache1 len=5 ['<e=5>', '<a=1>', '<d=4>', '<c=3>', '<b=2>']>


In [6]:
lru_cache1['f'] = 6
print(lru_cache1)  # last item 'b' will be popped

<LRUCache1 len=5 ['<f=6>', '<e=5>', '<a=1>', '<d=4>', '<c=3>']>


In [7]:
lru_cache1.pop('a')
print(lru_cache1)

<LRUCache1 len=4 ['<f=6>', '<e=5>', '<d=4>', '<c=3>']>


## 使用字典+双向链表实现

In [8]:
class Node(object):
    """双向链表中的节点"""

    def __init__(self, key, val, prev, post):
        self.key = key
        self.val = val
        self.prev = prev
        self.post = post

    def __repr__(self):
        return '<%s=%s>' % (self.key, self.val)

In [9]:
class LRUCacheV2(abc.MutableMapping):
    """使用字典+双向链表实现的LRU cache"""

    def __init__(self, size):
        self.size = size
        self._map = dict()  # 用于保存 kv
        self._len = 0
        self._head = Node(None, None, None, None)
        self._tail = Node(None, None, None, None)
        self._head.post = self._tail
        self._tail.prev = self._head

    def _add_first(self, node):
        node.prev = self._head
        node.post = self._head.post
        self._head.post.prev = node
        self._head.post = node

    def _move_to_first(self, node):
        self._del_node(node)
        self._add_first(node)

    def _del_last(self):
        last = self._tail.prev
        assert last is not self._head
        self._del_node(last)
        return last

    def _del_node(self, node):
        prev = node.prev
        post = node.post
        prev.post = post
        post.prev = prev
        node.prev = None
        node.post = None

    def __setitem__(self, key, value):
        assert value is not None, "value cannot be None"

        node = self._map.get(key)
        if node is None:
            # 新建 node 并插入到 head 后面
            node = Node(key, value, None, None)
            self._add_first(node)
            self._map[key] = node
            self._len += 1
            if self._len > self.size:
                last = self._del_last()
                self._map.pop(last.key)
                self._len -= 1
        else:
            # 更新 kv
            node.val = val
            self.move_to_first(node)

    def __getitem__(self, key):
        node = self._map.get(key)
        if node is None:
            return None

        self._move_to_first(node)
        return node.val

    def __delitem__(self, key):
        node = self._map.get(key)
        if node is None:
            return None

        self._del_node(node)
        self._map.pop(key)
        self._len -= 1
        return node.val

    def __len__(self):
        return self._len

    def __iter__(self):
        node = self._head.post
        while node is not self._tail:
            yield node
            node = node.post

    def __str__(self):
        return '<LRUCache2 len=%s %s>' % (self._len, list(self))

In [10]:
lru_cache2 = LRUCacheV2(5)
lru_cache2['a'] = 1
lru_cache2['b'] = 2
lru_cache2['c'] = 3
lru_cache2['d'] = 4

print('len=%s' % len(lru_cache2))
print(lru_cache2)

len=4
<LRUCache2 len=4 [<d=4>, <c=3>, <b=2>, <a=1>]>


In [11]:
print('a=%s' % lru_cache2['a']) # 'a' will be move to fist
print(lru_cache2)

a=1
<LRUCache2 len=4 [<a=1>, <d=4>, <c=3>, <b=2>]>


In [12]:
lru_cache2['e'] = 5
print(lru_cache2)  # meet max size=5

<LRUCache2 len=5 [<e=5>, <a=1>, <d=4>, <c=3>, <b=2>]>


In [13]:
lru_cache2['f'] = 6
print(lru_cache2)  # last 'b' will be removed

<LRUCache2 len=5 [<f=6>, <e=5>, <a=1>, <d=4>, <c=3>]>


In [14]:
lru_cache2.pop('a')
print(lru_cache2)

<LRUCache2 len=4 [<f=6>, <e=5>, <d=4>, <c=3>]>
